In [ ]:
from main import client, Part, analysis_prompt, recover_json, save_incident, db

gcs_uri = "gs://keyanvisionaibucket/visionintelligence/panic_test.mp4"
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        Part.from_uri(
            file_uri=gcs_uri,
            mime_type="video/mp4",
        ),
        analysis_prompt,
    ],
)

analysis_result = recover_json(response.text)
zone_id = "west-zone"
video_id = "123"
ts = "2025-07-25T00:00:00Z"
analysis_result
1/0.1
incident_ids = []
if 'incidents' in analysis_result:
    for incident_type, incident_data in analysis_result['incidents'].items():
        if incident_data.get('score', 0) > 0.5:
            incident_id = save_incident(incident_type, incident_data, zone_id, video_id, ts)
            incident_ids.append(incident_id)
# Save the overall analysis report
report_ref = db.collection('analysis_reports').document()
report_ref.set({
    "video_id": video_id,
    "video_uri": gcs_uri,
    "zone_id": zone_id,
    "created_at": ts,
    "overall_safety_assessment": analysis_result.get("overall_safety_assessment"),
    "recommended_actions": analysis_result.get("recommended_actions"),
    "incident_refs": [db.collection('incidents').document(id) for id in incident_ids]
})

print(f"Video analysis successful. Report ID: {report_ref.id}")
